In [1]:
import pandas as pd
import wbgapi as wb

In [46]:
country_codes = ['EGY','NGA','ZAF','CHN','IND','JPN','ITA','ESP','FRA','DEU']

In [79]:
indicator_ids = ["EN.ATM.CO2E.PC","AG.LND.AGRI.ZS","TX.VAL.FUEL.ZS.UN","SP.POP.GROW","AG.LND.ARBL.ZS","AG.LND.AGRI.ZS","AG.LND.FRST.ZS"]
indicator_dict = {
    "EN.ATM.CO2E.PC" : "co2_emission",
    "AG.LND.AGRI.ZS" : "agric_lands",
    "TX.VAL.FUEL.ZS.UN" : "fuel_export",
    "SP.POP.GROW" : "pop_growth",
    "AG.LND.ARBL.ZS" : "arable_land",
    "AG.LND.AGRI.ZS" : "agric_land",
    "AG.LND.FRST.ZS" : "forest_land",
    
}

In [73]:
wb.series.info(indicator_ids)

id,value
AG.LND.AGRI.ZS,Agricultural land (% of land area)
AG.LND.ARBL.ZS,Arable land (% of land area)
EN.ATM.CO2E.PC,CO2 emissions (metric tons per capita)
AG.LND.FRST.ZS,Forest area (% of land area)
TX.VAL.FUEL.ZS.UN,Fuel exports (% of merchandise exports)
SP.POP.GROW,Population growth (annual %)
,6 elements


In [76]:
# specicif time range
df = wb.data.DataFrame(indicator_ids, country_codes, time=range(2008, 2019))

In [98]:
country_df = df
country_df.reset_index()
country_df = country_df.reset_index(level=['economy', 'series'])
country_df = country_df.set_axis(['country','indicators','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018'], axis=1)
country_df = country_df.set_index('indicators')
country_df = country_df.rename(index=indicator_dict)
country_df = country_df.reset_index(level=['indicators'])
country_df = country_df.set_index(['country','indicators'])
country_df

2008       2009       2010       2011       2012  \
country indicators                                                            
CHN     agric_land    56.070937  56.133200  56.120276  56.116911  56.107828   
        arable_land   12.922614  12.942968  12.877841  12.800417  12.728202   
        forest_land   20.784578  21.035087  21.285597  21.491096  21.696596   
        co2_emission   5.431610   5.795344   6.332166   6.898148   7.039788   
        pop_growth     0.512387   0.497381   0.482960   0.546458   0.678345   
        fuel_export    2.220806   1.696237   1.690563   1.700077   1.513750   
DEU     agric_land    48.535697  48.438083  47.910032  47.965917  47.810868   
        arable_land   34.225397  34.264651  33.984566  34.068740  33.953061   
        forest_land   32.693687  32.711339  32.730872  32.737549  32.745166   
        co2_emission   9.615748   8.969955   9.451565   9.297791   9.449950   
        pop_growth    -0.190143  -0.253383  -0.153198  -1.853715   0.187728   
        fuel_export    2.614730   2.025758   1.967774   2.296469   2.442987   
EGY     agric_land     3.558190   3.705862   3.687779   3.636546   3.712894   
        arable_land    2.654076   2.897182   2.886132   2.820835   2.841931   
        forest_land    0.064648   0.065294   0.065940   0.062480   0.059021   
        co2_emission   2.374803   2.409447   2.374300   2.385092   2.457816   
        pop_growth     1.778691   1.864456   1.984805   2.113776   2.214741   
        fuel_export   44.413499  28.469542  28.675015  29.250450  30.129061   
ESP     agric_land    56.419407  56.090322  55.088898  54.040970  53.861378   
        arable_land   25.036087  25.061164  25.055499  24.863967  25.129446   
        forest_land   36.597951  36.899288  37.089938  37.101920  37.079778   
        co2_emission   7.053994   6.200857   5.866642   5.870436   5.756947   
        pop_growth     1.595330   0.885736   0.460408   0.355338   0.064926   
        fuel_export    6.611120   4.607065   5.218536   6.349594   7.661606   
FRA     agric_land    53.163598  52.998866  52.827925  52.740811  52.679082   
        arable_land   33.364015  33.341186  33.422456  33.374425  33.387757   
        forest_land   29.572812  29.779365  29.985919  30.138232  30.290545   
        co2_emission   5.564429   5.314414   5.350659   5.104312   5.122311   
        pop_growth     0.558843   0.514490   0.494041   0.483676   0.483999   
        fuel_export    5.069604   3.531343   3.658113   4.543182   4.446403   
IND     agric_land    60.459305  60.563906  60.397418  60.430043  60.420626   
        arable_land   53.139894  53.116013  52.808263  52.798173  52.652538   
        forest_land   23.246076  23.310148  23.374221  23.463822  23.553422   
        co2_emission   1.185905   1.288081   1.349214   1.408316   1.507821   
        pop_growth     1.464890   1.410583   1.350338   1.288513   1.231485   
        fuel_export   18.064889  13.586405  17.229655  18.758546  18.778894   
ITA     agric_land    49.150745  47.536547  48.710818  47.095261  46.675053   
        arable_land   25.491263  23.733596  23.940980  22.846264  24.199361   
        forest_land   30.245060  30.469032  30.693003  30.875937  31.058870   
        co2_emission   7.564248   6.718970   6.836836   6.680594   6.327709   
        pop_growth     0.662469   0.455613   0.307591   0.171978   0.269541   
        fuel_export    4.612586   3.557456   4.638894   4.760732   4.699485   
JPN     agric_land    12.696845  12.644719  12.600823  12.513032  12.480110   
        arable_land   11.818930  11.780521  11.747599  11.670782  11.648834   
        forest_land   68.444444  68.469136  68.493827  68.481756  68.469684   
        co2_emission   9.055074   8.600045   9.026938   9.482450   9.811406   
        pop_growth     0.048425  -0.012495   0.017961  -0.185226  -0.159711   
        fuel_export    2.402803   1.813317   1.695026   1.979308   1.682145   
NGA     agric_land    74.102581  74.281887  74.461192  74.640498  74.819803  

In [220]:
years_df = country_df.reset_index(level=['country','indicators'])
years_df = years_df.set_index(['indicators','country'])
years_df = years_df.stack()
years_df = years_df.unstack('country')
years_df = years_df.swaplevel()
years_df = years_df.sort_index()
# years_df.head()
years_df.loc[("2008","arable_land"):("2010","fuel_export")]

country                  CHN        DEU        EGY        ESP        FRA  \
     indicators                                                            
2008 arable_land   12.922614  34.225397   2.654076  25.036087  33.364015   
     co2_emission   5.431610   9.615748   2.374803   7.053994   5.564429   
     forest_land   20.784578  32.693687   0.064648  36.597951  29.572812   
     fuel_export    2.220806   2.614730  44.413499   6.611120   5.069604   
     pop_growth     0.512387  -0.190143   1.778691   1.595330   0.558843   
2009 agric_land    56.133200  48.438083   3.705862  56.090322  52.998866   
     arable_land   12.942968  34.264651   2.897182  25.061164  33.341186   
     co2_emission   5.795344   8.969955   2.409447   6.200857   5.314414   
     forest_land   21.035087  32.711339   0.065294  36.899288  29.779365   
     fuel_export    1.696237   2.025758  28.469542   4.607065   3.531343   
     pop_growth     0.497381  -0.253383   1.864456   0.885736   0.514490   
2010 agric_land    56.120276  47.910032   3.687779  55.088898  52.827925   
     arable_land   12.877841  33.984566   2.886132  25.055499  33.422456   
     co2_emission   6.332166   9.451565   2.374300   5.866642   5.350659   
     forest_land   21.285597  32.730872   0.065940  37.089938  29.985919   
     fuel_export    1.690563   1.967774  28.675015   5.218536   3.658113   

country                  IND        ITA        JPN        NGA        ZAF  
     indicators                                                           
2008 arable_land   53.139894  25.491263  11.818930  39.526994  10.551567  
     co2_emission   1.185905   7.564248   9.055074   0.609105   8.568994  
     forest_land   23.246076  30.245060  68.444444  25.897419  14.415163  
     fuel_export   18.064889   4.612586   2.402803  91.743289   9.626525  
     pop_growth     1.464890   0.662469   0.048425   2.649868   1.334117  
2009 agric_land    60.563906  47.536547  12.644719  74.281887  79.951199  
     arable_land   53.116013  23.733596  11.780521  35.135105  10.436159  
     co2_emission   1.288081   6.718970   8.600045   0.481063   8.004039  
     forest_land   23.310148  30.469032  68.469136  25.718113  14.385157  
     fuel_export   13.586405   3.557456   1.813317  90.362190  11.181294  
     pop_growth     1.410583   0.455613  -0.012495   2.662921   1.391535  
2010 agric_land    60.397418  48.710818  12.600823  74.461192  79.871238  
     arable_land   52.808263  23.940980  11.747599  36.233078  10.331468  
     co2_emission   1.349214   6.836836   9.026938   0.572417   8.300179  
     forest_land   23.374221  30.693003  68.493827  25.538808  14.355151  
     fuel_export   17.229655   4.638894   1.695026  87.132640  10.710234